In [12]:
import numpy as np
import gymnasium as gym
import games

In [5]:
env = gym.make('games/TicTacToe')

In [ ]:
class PolicyValueNet:
    def forward(state):
        pass
    

In [ ]:
class AgentNode:
    """
    Each state_action pair (s, a) stores a set of statistics, {N(s, a), W(s, a), Q(s, a), P(s, a)},
    where N(s, a) is the visit count, W(s, a) is the mean action-value, and P(s, a) is the prior probability
    of selecting a in s.
    """
    def __init__(self, parent, N, W, Q, P):
        self.parent = parent
        self.N = N
        self.W = W
        self.Q = Q
        self.P = P

    def select(self, state):
        next_prior_p, state_value = PolicyValueNet.forward(state)
        upper_confs = self.prior_p
        pass




In [64]:
class Agent:
    def __init__(self, mark, action_space, q_function):
        self.mark = mark
        self.action_space = action_space
        self.q_function = q_function
        
    def set_mark(self, mark):
        self.mark = mark

    def play(self, state):
        q_values = None
        return np.random.randint(self.action_space.start, self.action_space.n), q_values

In [65]:
env = gym.make('games/TicTacToe')
action_space = env.unwrapped.action_space
agent_index_space = env.unwrapped.agent_index_space
agent_mark_mapping = env.unwrapped.agent_mark_mapping

agent_list = [Agent(mark=agent_mark_mapping[i], action_space=action_space) for i in range(agent_index_space.start, agent_index_space.n)]

In [69]:
observation, info = env.reset()
is_end = False
while not is_end:
    print(observation)
    agent = agent_list[info['agent_index']]
    action, q_values = agent.play(observation)
    observation, reward, terminated, truncated, info = env.step(action)
    is_end = terminated or truncated

[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 2]]
[[0 0 1]
 [0 0 0]
 [0 0 2]]
[[2 0 1]
 [0 0 0]
 [0 0 2]]
[[2 0 1]
 [0 0 1]
 [0 0 2]]
[[2 0 1]
 [0 0 1]
 [0 2 2]]
[[2 0 1]
 [1 0 1]
 [0 2 2]]
